# Bot-E Image Meme Generation

This notebook currently focuses on creating memes for bot-e questions. Instagram requires jpg format, so this script converts the png file from the question into a jpg file for the meme.

Meme Creation Steps:

- Get url for question image you would like to create a meme for.
- Toggle settings until the meme text looks good
- post meme image to server (TODO)
- The server will post 2 memes per hour max to Meta



In [49]:
from PIL import Image, ImageDraw, ImageFont
import os
import requests
from io import BytesIO
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import ipywidgets as widgets
from IPython.display import display, clear_output

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

def hex_to_rgb(value):
    value = value.lstrip('#')
    length = len(value)
    return tuple(
        int(value[i:i + length // 3], 16) for i in range(0, length, length // 3)
    )

def create_meme(image_url, font_path, text1, text2=None, position='bottom', alignment='right', font_size=40, font_color='#FFFFFF', horizontal_margin=10, vertical_margin=10):
    try:
        response = requests.get(image_url, verify=False, timeout=10)  
        response.raise_for_status()  
    except requests.RequestException as e:
        print(f"Failed to retrieve the image: {e}")
        return

    img_data = BytesIO(response.content)
    img = Image.open(img_data).convert("RGBA")  
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype(font_path, font_size)

    text_color = hex_to_rgb(font_color)

    img_width, img_height = img.size

    # Define the dimensions of the textbox
    textbox_width = img_width - 2 * horizontal_margin
    textbox_height = img_height - 2 * vertical_margin

    # Calculate the position of the text within the textbox
    if position == 'top':
        y = vertical_margin
    else:
        y = img_height - textbox_height - vertical_margin

    if alignment == 'left':
        x = horizontal_margin
    elif alignment == 'center':
        x = (img_width - textbox_width) / 2
    else:
        x = img_width - textbox_width - horizontal_margin

    # Create a function to draw text within a textbox
    def draw_text_with_textbox(text, x, y):
        nonlocal draw
        text_bbox = draw.textbbox((x, y), text, font=font)
        text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]
        x = x + (textbox_width - text_width) / 2
        y = y + (textbox_height - text_height) / 2
        draw.text((x, y), text, font=font, fill=text_color)

    # Draw the first text within the textbox
    draw_text_with_textbox(text1, x, y)

    if text2:
        # Move down for the second line of text
        y += textbox_height - 15
        # Draw the second text within the same textbox
        draw_text_with_textbox(text2, x, y)

    # Convert the RGBA image to RGB before saving as JPEG
    rgb_img = img.convert("RGB")

    rgb_img.save('images/memes/meme.jpg')  # Save as JPEG
    rgb_img.show()

# List all files in the fonts/ directory
font_files = [f for f in os.listdir('fonts/') if f.endswith('.ttf')]

# Create a Dropdown widget with the font files
font_path_widget = widgets.Dropdown(
    options=[os.path.join('fonts/', f) for f in font_files],
    description='Font:',
)
# Define a layout for wider fields
wide_layout = Layout(width='500px')

# Now apply this layout to your widgets
image_url_widget = widgets.Text(value='https://bot-e.com/images/questions/f/fUakuRPLjyr.png', description='Image URL:', layout=wide_layout)
font_path_widget = widgets.Text(value='fonts/Lato-Regular.ttf', description='Font Path:', layout=wide_layout)
text1_widget = widgets.Text(value='I Love', description='Text 1:', layout=wide_layout)
text2_widget = widgets.Text(value='Frogs', description='Text 2:', layout=wide_layout)
position_widget = widgets.Dropdown(options=['top', 'bottom'], value='top', description='Position:')
alignment_widget = widgets.Dropdown(options=['left', 'center', 'right'], value='left', description='Alignment:')
font_size_widget = widgets.IntSlider(value=20, min=10, max=100, description='Font Size:')
font_color_widget = widgets.ColorPicker(value='#FFFFFF', description='Font Color:')
horizontal_margin_widget = widgets.IntSlider(value=30, min=0, max=100, description='Horiz. Margin:')
vertical_margin_widget = widgets.IntSlider(value=280, min=0, max=500, description='Vert. Margin:')

# Define a button to trigger the function
run_button = widgets.Button(description='Create Meme')

# Define the output widget
output_widget = widgets.Output()

# Define the function to be triggered
def on_button_clicked(b):
    with output_widget:
        clear_output()
        create_meme(
            image_url=image_url_widget.value,
            font_path=font_path_widget.value,
            text1=text1_widget.value,
            text2=text2_widget.value,
            position=position_widget.value,
            alignment=alignment_widget.value,
            font_size=font_size_widget.value,
            font_color=font_color_widget.value,
            horizontal_margin=horizontal_margin_widget.value,
            vertical_margin=vertical_margin_widget.value
        )

# Link the button click event to the function
run_button.on_click(on_button_clicked)

# Display the widgets
widgets.VBox([
    image_url_widget,
    font_path_widget,
    text1_widget,
    text2_widget,
    position_widget,
    alignment_widget,
    font_size_widget,
    font_color_widget,
    horizontal_margin_widget,
    vertical_margin_widget,
    run_button,
    output_widget
])



# Panel meme

this version creates a colored panel for 3 lines of text.


In [54]:
from PIL import Image, ImageDraw, ImageFont
import requests
from io import BytesIO
import os

def hex_to_rgb(value):
    value = value.lstrip('#')
    length = len(value)
    return tuple(
        int(value[i:i + length // 3], 16) for i in range(0, length, length // 3)
    )

def create_meme(code, font_path, text1, text2=None, text3=None, position='bottom', font_size=40, font_color='#FFFFFF', rect_color='#000000', horizontal_margin=10, vertical_margin=10, signature=None, local_image=None):
    if local_image is None:
        image_url = f'https://bot-e.com/images/questions/{code[0].lower()}/{code}.png'

        try:
            response = requests.get(image_url, verify=False, timeout=10)  
            response.raise_for_status()  
        except requests.RequestException as e:
            print(f"Failed to retrieve the image: {e}")
            return

        img_data = BytesIO(response.content)
        img = Image.open(img_data).convert("RGBA")  
    else:
        img = Image.open(local_image).convert("RGBA")

    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype(font_path, font_size)

    text_color = hex_to_rgb(font_color)
    rect_fill_color = hex_to_rgb(rect_color)

    img_width, img_height = img.size

    # Calculate the position and dimensions of the rectangle
    rect_height = img_height // 5
    if position == 'top':
        rect_y = 0
    else:
        rect_y = img_height - rect_height

    # Draw the rectangle
    draw.rectangle([0, rect_y, img_width, rect_y + rect_height], fill=rect_fill_color)

    # Calculate center positions for the text
    def center_text(text):
        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width, text_height = text_bbox[2], text_bbox[3]
        x = (img_width - text_width) / 2
        return text_width, text_height, x

    text_width1, text_height1, x1 = center_text(text1)
    text_width2, text_height2, x2 = center_text(text2) if text2 else (0, 0, 0)
    text_width3, text_height3, x3 = center_text(text3) if text3 else (0, 0, 0)

    if position == 'top':
        y1 = vertical_margin
        y2 = y1 + text_height1
        y3 = y2 + text_height2
    else:
        y3 = img_height - vertical_margin - text_height3
        y2 = y3 - text_height2
        y1 = y2 - text_height1

    draw.text((x1, y1), text1, font=font, fill=text_color)
    if text2:
        draw.text((x2, y2), text2, font=font, fill=text_color)
    if text3:
        draw.text((x3, y3), text3, font=font, fill=text_color)
    
    # Draw signature
    if signature:
        sig_bbox = draw.textbbox((0, 0), signature, font=font)
        sig_width, sig_height = sig_bbox[2], sig_bbox[3]
        sig_x = img_width - sig_width - horizontal_margin - 50 # Offset by 10 to leave a right margin
        sig_y = y3 + text_height3 + 0  # Position below the 3rd line, offset by 10
        draw.text((sig_x, sig_y), signature, font=font, fill=text_color)

    # Convert the RGBA image to RGB before saving as JPEG
    rgb_img = img.convert("RGB")

    # Ensure the directory exists
    os.makedirs('images/memes', exist_ok=True)

    # Save as JPEG
    file_path = f'images/memes/{code}.jpg'
    rgb_img.save(file_path)
    rgb_img.show()
    print(f'Meme saved at: {file_path}')

# Sample call to the function with a local image
create_meme(
    code='sYsmUJYUhVP',
    font_path='fonts/TitilliumWeb-Regular.ttf',
    text1='The wear and tear on your corporeal',
    text2="form is as alian to me as the",
    text3='concept of a suntan.',
    position='bottom',
    font_size=18,
    font_color='#FFFFFF',
    rect_color='#2E4053',
    horizontal_margin=0,
    vertical_margin=30,
    signature='- Bot-E',
    #local_image='images/memes/batch_process.png'  # Provide the path to your local image here
)


Meme saved at: images/memes/sYsmUJYUhVP.jpg


# Create image with stability.ai

Create a new image with the stability.ai api.  This is used to update the existing image of a question.

In [30]:
import ipywidgets as widgets
from IPython.display import display
from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation
from PIL import Image
import io
import os

os.environ["STABILITY_HOST"] = "grpc.stability.ai:443"

def stability_image(prompt, file_name):
    
    stability_api = client.StabilityInference(
        key=os.environ["STABILITY_KEY"],
        verbose=True,  # Print debug messages.
        engine="stable-diffusion-xl-1024-v1-0",
    )

    answers = stability_api.generate(
        prompt=prompt,
        seed=0,
        steps=30,
        cfg_scale=3.0,
        width=1024,
        height=1024,
        samples=1,
        sampler=generation.SAMPLER_K_DPMPP_2M,
        #sampler=generation.SAMPLER_DDIM,
        #sampler=generation.SAMPLER_K_EULER,
        #sampler=generation.SAMPLER_K_EULER_ANCESTRAL,
        #sampler=generation.SAMPLER_K_HEUN,
        #sampler=generation.SAMPLER_K_DPM_2,
        #sampler=generation.SAMPLER_K_LMS,
        #sampler=generation.SAMPLER_K_DPMPP_SDE,
        
    )

    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.finish_reason == generation.FILTER:
                raise ValueError(
                    "Your request activated the API's safety filters and could not be processed."
                )
            if artifact.type == generation.ARTIFACT_IMAGE:
                img = Image.open(io.BytesIO(artifact.binary))
                image_filename = f"images/memes/{file_name}.png"
                img.save(image_filename)
                img.show()
                first_char = file_name[0].lower()
                scp_command = (
                    f"scp -o 'ProxyJump=culley@spinningzoo.com' "
                    f"{file_name}.png "
                    f"culley@dt20-pdx.downloadtech.net:/usr/local/bot-e/images/questions/{first_char}/"
                )
                #print(scp_command)
                return scp_command

    return ""

prompt = "profile picture for a humanoid robot"
file_name = "profile"
scp_command = stability_image(prompt, file_name)

print(scp_command)





scp -o 'ProxyJump=culley@spinningzoo.com' profile.png culley@dt20-pdx.downloadtech.net:/usr/local/bot-e/images/questions/p/


# Create stability.ai animation

w.i.p.

In [3]:
from stability_sdk import api
import os
from stability_sdk.animation import AnimationArgs, Animator
from stability_sdk.utils import create_video_from_frames
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation
from tqdm import tqdm


STABILITY_HOST = "grpc.stability.ai:443"
STABILITY_KEY = os.environ["STABILITY_KEY"]


context = api.Context(STABILITY_HOST, STABILITY_KEY)

# Configure the animation
args = AnimationArgs()
args.interpolate_prompts = True
args.locked_seed = True
args.max_frames = 48
args.init_image='images/memes/LQL12hjNoRL.png'
args.seed = -1
#sampler=generation.SAMPLER_K_DPM_2_ANCESTRAL
#cfg_scale=24.0
#args.strength_curve = "0:(0)"
#args.diffusion_cadence_curve = "0:(4)"
args.cadence_interp = "film"

animation_prompts = {
    0: "the lenses of the sunglasses change color",
    
}
negative_prompt = ""

# Create Animator object to orchestrate the rendering
animator = Animator(
    api_context=context,
    animation_prompts=animation_prompts,
    negative_prompt=negative_prompt,
    args=args,
    out_dir='images/animations'
)

# Render each frame of animation
#for idx, frame in enumerate(animator.render()):
#    frame.save(f"images/animations/frame_{idx:05d}.png")


for _ in tqdm(animator.render(), total=args.max_frames):
    pass

create_video_from_frames(animator.out_dir, "video.mp4", fps=24)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [05:38<00:00,  7.06s/it]


In [4]:
print("test");


test
